In [ ]:
# coding: utf-8
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = ""

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
 
base_model = VGG19(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
img_path = 'content/0.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
block5_pool_features = model.predict(x)
print(block5_pool_features.shape) #(1, 7, 7, 512)


## build the transfer model

In [ ]:

MEAN_VALUES = np.array([123.68, 116.779, 103.939]).reshape((1, 1, 1, 3)) # 通道颜色均值
#k_initializer = tf.truncated_normal_initializer(0, 0.1)
k_initializer = keras.initializers.TruncatedNormal(0, 0.1)

def relu(X):
    return keras.layers.Activation('relu')(X)

def instance_norm(X):
    return keras.layers.LayerNormalization()(X)

def conv2d(inputs, filters, kernel_size, strides, name = "noname"):
    return keras.layers.Conv2D(filters, kernel_size, strides, padding="same", kernel_initializer=k_initializer, name = name)(inputs)

def deconv2d(inputs, filters, kernel_size = 3, strides = 1, name="noname"):
    # shape = tf.shape(inputs)
    # height, width = shape[1], shape[2]
    # # 近邻插值法，
    # X = keras.backend.resize_images(inputs, height*strides*2, width*strides*2, "channels_last", 'nearest')

    #X = tf.image.resize(inputs, [height * strides * 2, width * strides * 2], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    #return conv2d(X, filters, kernel_size, strides, name=name)
    #filters, kernel_size, strides, padding="same", kernel_initializer=k_initializer, name = name)(inputs)
    return tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides, name=name, padding="same", dilation_rate=2, kernel_initializer=k_initializer)(inputs)

    # 残差网络
def residual(inputs, filters = 128, kernel_size = 3, name="noname"):
    X = relu(conv2d(inputs, filters, kernel_size, 1, name=name+"_1"))
    X = conv2d(X, filters, kernel_size, 1, name=name+"_2")
    return keras.layers.Add()([inputs, X])

def transfer_model(input_shape=(256, 256, 3)):
    img_inputs = keras.Input(input_shape, name="transfer_inputs")
    #X = tf.pad(img_inputs - MEAN_VALUES, [[0, 0], [10, 10], [10, 10], [0, 0]], mode='reflect')
    X = keras.layers.Subtract()([img_inputs, MEAN_VALUES])
    X = relu(instance_norm(conv2d(X, 32, 9, 1, name="conv1")))
    X = relu(instance_norm(conv2d(X, 64, 3, 2, name="conv2")))
    X = relu(instance_norm(conv2d(X, 128, 3, 2, name="conv3")))

    for i in range(5):
        X = residual(X, 128, 3, name="res"+str(i))

    X = relu(instance_norm(deconv2d(X, 64, 3, 2, name="conv4")))
    X = relu(instance_norm(deconv2d(X, 32, 3, 2, name="conv5")))
    X = tf.nn.tanh(instance_norm(conv2d(X, 3, 9, 1, name="conv6")))
    #X = (X + 1) * (255.0/2)
    X = keras.layers.Lambda(lambda x: (x+1)*(255.0/2), name="transfer_outputs")(X)
    #X = keras.layers.Multiply()([X+1, 255.0/2], name="transfer_outputs")
    #shape = tf.shape(X)
    #generates = tf.slice(X, [0, 10, 10, 0], [-1, shape[1] - 20, shape[2] - 20, -1], name='generate')
    return img_inputs, X

In [ ]:
img_inputs, img_outputs = transfer_model()
model = keras.Model(inputs=img_inputs, outputs=img_outputs, name="transfer")
model.summary()

In [ ]:
keras.utils.plot_model(model , show_shapes=True)